In [1]:
import pandas as pd
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo

In [5]:
pd.read_csv('data/games.csv').tail(50)

,game_id,Date,Time,Home,Away,event_id,insert_timestamp_utc
32,202411240SAC,2024-11-24,2024-11-24 21:00:00,SAC,BRK,NaN,2024-11-24 14:14:59.018700+00:00
33,202411250CHO,2024-11-25,2024-11-25 19:00:00,CHO,ORL,NaN,2024-11-25 14:16:47.498719+00:00
34,202411250DET,2024-11-25,2024-11-25 19:00:00,DET,TOR,NaN,2024-11-25 14:16:47.498719+00:00
35,202411250IND,2024-11-25,2024-11-25 19:00:00,IND,NOP,NaN,2024-11-25 14:16:47.498719+00:00
36,202411250ATL,2024-11-25,2024-11-25 19:30:00,ATL,DAL,NaN,2024-11-25 14:16:47.498719+00:00
37,202411250BOS,2024-11-25,2024-11-25 19:30:00,BOS,LAC,NaN,2024-11-25 14:16:47.498719+00:00
38,202411250MEM,2024-11-25,2024-11-25 20:00:00,MEM,POR,NaN,2024-11-25 14:16:47.498719+00:00
39,202411250DEN,2024-11-25,2024-11-25 21:00:00,DEN,NYK,NaN,2024-11-25 14:16:47.498719+00:00
40,202411250GSW,2024-11-25,2024-11-25 22:00:00,GSW,BRK,NaN,2024-11-25 14:16:47.498719+00:00
41,202411250SAC,2024-11-25,2024-11-25 22:00:00,SAC,OKC,NaN,2024-11-25 14:16:47.498719+00:00


In [14]:
import requests
from bs4 import BeautifulSoup
import pytz
import os
import json

season = 2025
today = datetime.now(ZoneInfo('America/New_York')) + timedelta(days = 1)
tmrw_utc = (today + timedelta(days = 1)).astimezone(pytz.utc)


url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games-{today.strftime("%B").lower()}.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')
table = soup.find('table')
while table.find_all('tr', class_ = 'thead') :
    table.find('tr', class_ = 'thead').decompose()
games = pd.read_html(str(table))[0].rename(columns = {'Start (ET)': 'Time'})
games['Home'] = [x['href'].split('/')[2] for x in table.find_all('a', href = True) if 'teams' in x['href']][1::2]
games['Away'] = [x['href'].split('/')[2] for x in table.find_all('a', href = True) if 'teams' in x['href']][0::2]
games['Date'] = pd.to_datetime(games['Date'])
games = games[pd.to_datetime(games['Date']) == pd.to_datetime(today.date())].reset_index(drop = True)
games['Time'] = (games['Date'].astype(str) + ' ' +  games['Time']).apply(lambda x: datetime.strptime(x.upper() + 'M', "%Y-%m-%d %I:%M%p"))
games['game_id'] = games['Date'].apply(lambda x: datetime.strftime(x, "%Y%m%d")) + '0' + games['Home']
games = games[['game_id', 'Date', 'Time', 'Home', 'Away']]

/var/folders/yb/vy0d2g3s36j5mzjxqc1kwwpw0000gr/T/ipykernel_15855/2991714655.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  games = pd.read_html(str(table))[0].rename(columns = {'Start (ET)': 'Time'})


In [15]:
if os.getenv("GITHUB_ACTIONS") == "true" :
  api_key = os.getenv('ODDS_API_KEY')
else :
  with open('secrets/odds_api_key.txt') as f:
    api_key = f.read()

with open('utils/odds_tm_map.json', 'r') as f :
  odds_tm_map = json.load(f)

events_response = requests.get(f'https://api.the-odds-api.com/v4/sports/basketball_nba/events',
                               params = {'apiKey': api_key,
                                         'commenceTimeTo': tmrw_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})


assert events_response.status_code == 200, f'Odds API query not successful {events_response.status_code}'

# Map odds API event_id to game_id
odds = pd.DataFrame(events_response.json()).rename(columns = {'id': 'event_id'})
odds['game_id'] = today.strftime('%Y%m%d') + '0' + odds['home_team'].map(odds_tm_map)

games = games.merge(
    odds[['game_id', 'event_id']],
    on = 'game_id',
    how = 'left'
)

games['insert_timestamp_utc'] = datetime.now(timezone.utc)

In [ ]:
# Store subset of games in the next 30 minutes
now = datetime.now(ZoneInfo('America/New_York'))
now = datetime(2024, 12, 1, 15, 2)

games_now = games[(games['Time'] > now) & (games['Time'] <= now + timedelta(minutes = 30))]

# Iterate through games
for _, game in games_now.iterrows() :

    eventId = game['event_id']

    odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/basketball_nba/events/{eventId}/odds',
                             params = {'apiKey': api_key,
                                       'regions': 'us',
                                       'markets': 'player_first_basket',
                                       'oddsFormat': 'decimal'})
    

    bm_dfs = [pd.DataFrame(columns = ['name', 'price', 'bookmaker', 'update_time'])]
    for bookmaker in odds_response.json()['bookmakers'] :
        
        bm_df = pd.DataFrame(bookmaker['markets'][0]['outcomes'])
        bm_df['bookmaker'] = bookmaker['key']
        bm_df['update_time'] = bookmaker['markets'][0]['last_update']

        bm_dfs.append(
            bm_df
            .drop(columns = 'name')
            .rename(columns = {'description': 'name'})
        )

    game_df = pd.concat(bm_dfs).reset_index(drop = True)
    game_df['game_id'] = game['game_id']
    game_df['event_id'] = game['event_id']

    game_df['insert_timestamp_utc'] = datetime.now(timezone.utc)
    game_df.to_csv('data/odds_first_basket.csv', index = None, header = None, mode = 'a')

ValueError: No objects to concatenate

In [75]:
test = pd.read_csv('data/odds_first_basket.csv')[['price']]
test['prob'] = 10 * [0.1]

In [71]:
bm_dfs = [pd.DataFrame(columns = ['name', 'price', 'bookmaker', 'update_time'])]
for bookmaker in odds_response.json()['bookmakers'] :
    
    bm_df = pd.DataFrame(bookmaker['markets'][0]['outcomes'])
    bm_df['bookmaker'] = bookmaker['key']
    bm_df['update_time'] = bookmaker['markets'][0]['last_update']

    bm_dfs.append(
        bm_df
        .drop(columns = 'name')
        .rename(columns = {'description': 'name'})
    )

game_df = pd.concat(bm_dfs).reset_index(drop = True)
game_df['game_id'] = game['game_id']
game_df['event_id'] = game['event_id']

game_df['insert_timestamp_utc'] = datetime.now(timezone.utc)
game_df.to_csv('data/odds_first_basket.csv', index = None, header = None, mode = 'a')

In [85]:
import numpy as np

In [96]:
s = 1
lmbda = 1

test = pd.read_csv('data/odds_first_basket.csv')[['price']]
test['prob'] = 1 / test['price'] * np.random.uniform(0.5, 1.5, (10,))
test['prob'] /= test['prob'].sum()
test['E'] = (test['price'] * test['prob'] - 1) * s
test['var'] = (s ** 2) * (test['prob'] * ((test['price'] - 1) ** 2) + 1 - test['prob'] - (test['prob'] * test['price'] - 1) ** 2)
test['obj'] = test['E'] + lmbda * test['var']
test

,price,prob,E,var,obj
0,5.6,0.136351,-0.236435,3.692934,3.456499
1,7.0,0.179002,0.253011,7.201038,7.454049
2,8.5,0.073685,-0.373680,4.931443,4.557763
3,9.0,0.126602,0.139419,8.956494,9.095913
4,9.0,0.109996,-0.010033,7.929672,7.919639
5,9.5,0.079522,-0.244543,6.606128,6.361585
6,10.0,0.062705,-0.372955,5.877266,5.504311
7,10.0,0.054153,-0.458466,5.122077,4.663611
8,10.5,0.058793,-0.382673,6.100837,5.718164
9,11.0,0.119192,0.311109,12.703188,13.014297


$x_i \in \{0,1\} \ \forall \ i$

$E_i = P_i o_i s_i - s_i$